### Tính mean/std cho toàn bộ dataset

In [11]:
# !python "/home/serverai/ltdoanh/Motion_Diffusion/datasets/step1_fit_scaler.py" --parent-dir "{base_dir}" --start 1 --end 30 --mode hybrid

In [12]:
import sys
import joblib
import numpy as np
PYOM_DIR = "/home/serverai/ltdoanh/Motion_Diffusion/datasets/pymo"
if PYOM_DIR not in sys.path:
    sys.path.insert(0, PYOM_DIR)
from pymo.parsers import BVHParser

try:
    pipeline = joblib.load('global_pipeline.pkl')
    print(" Pipeline loaded successfully.")
except FileNotFoundError:
    print(" Error: 'global_pipeline.pkl' not found.")
    exit()

print("-" * 40)
print("Pipeline Steps Inspection:")
print("-" * 40)

found_downsampler = False

for name, step in pipeline.steps:
    step_type = str(type(step))
    print(f"Step: '{name}' -> Type: {step_type}")
    
    if 'DownSampler' in step_type:
        found_downsampler = True
        print(f"\n  FOUND DownSampler object!")
        print(f"   Listing all attributes of this object:")
        
        for attr, value in step.__dict__.items():
            if not attr.startswith('_'): 
                print(f"    {attr}: {value}")
                
        if hasattr(step, 'rate'):
            print(f"\n    Analysis: Found 'rate' = {step.rate}")
            print(f"   Nếu dữ liệu gốc là 120 FPS, thì Target FPS = 120 / {step.rate} = {120/step.rate}")
        elif hasattr(step, 'target_fps'):
             print(f"\n    Analysis: Found 'target_fps' = {step.target_fps}")

print("-" * 40)

bvh_file = "./datasets/BEAT/10/10_kieks_0_1_1.bvh"
import os
if os.path.exists(bvh_file):
    print(f"Testing on file: {bvh_file}")
    parser = BVHParser()
    data = parser.parse(bvh_file)
    print(f"Original frames: {data.values.shape[0]}")
    processed = pipeline.transform([data])
    new_frames = processed[0].shape[0]
    ratio = data.values.shape[0] / new_frames
    print(f"Reduction Ratio: {ratio}")
    print(f"==> ESTIMATED FPS (Assuming 120Hz input): {120 / ratio}")
else:
    print("   Skipped file test (path not set or file not found).")
    print("   Please rely on the 'Attribute' info above.")

 Pipeline loaded successfully.
----------------------------------------
Pipeline Steps Inspection:
----------------------------------------
Step: 'param' -> Type: <class 'pymo.preprocessing.MocapParameterizer'>
Step: 'rcpn' -> Type: <class 'pymo.preprocessing.RootCentricPositionNormalizer'>
Step: 'delta' -> Type: <class 'pymo.preprocessing.RootTransformer'>
Step: 'const' -> Type: <class 'pymo.preprocessing.ConstantsRemover'>
Step: 'np' -> Type: <class 'pymo.preprocessing.Numpyfier'>
Step: 'down' -> Type: <class 'pymo.preprocessing.DownSampler'>

  FOUND DownSampler object!
   Listing all attributes of this object:
    rate: 2

    Analysis: Found 'rate' = 2
   Nếu dữ liệu gốc là 120 FPS, thì Target FPS = 120 / 2 = 60.0
Step: 'stdscale' -> Type: <class 'pymo.preprocessing.ListStandardScaler'>
----------------------------------------
   Skipped file test (path not set or file not found).
   Please rely on the 'Attribute' info above.


### Chuyển dataset bvh sang npy theo từng segment

In [13]:
# import os
# for i in range(1, 31):
#     print("Đang chạy preprocess_data.py...")
#     command2 = f'python "/home/serverai/ltdoanh/Motion_Diffusion/datasets/preprocess_data.py" --parent-dir "{base_dir}" --out-root "{out_dir}" --folders "{i}" --fps 60'
#     os.system(command2)

In [14]:
# !python "/home/serverai/ltdoanh/Motion_Diffusion/datasets/preprocess_data.py" --parent-dir "{base_dir}" --out-root "{out_dir}" --folders "5"

In [15]:
# from textgrid import TextGrid

# tg = TextGrid.fromFile("/home/serverai/ltdoanh/Motion_Diffusion/datasets/BEAT/5/5_stewart_0_82_82.TextGrid")
# print(f"Number of tiers: {len(tg)}")
# for i, tier in enumerate(tg):
#     print(f"Tier {i}: {tier.name}, intervals: {len(tier.intervals)}")

In [1]:
import numpy as np
import os
import random

# Đường dẫn gốc chứa các thư mục con 1, 2, ..., 30
base_path = "/home/serverai/ltdoanh/Motion_Diffusion/datasets/BEAT_numpy/npy"

# Lấy danh sách tất cả các thư mục con (1, 2, 3...)
sub_dirs = sorted([d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))], key=lambda x: int(x) if x.isdigit() else 0)

print(f"Tìm thấy {len(sub_dirs)} thư mục con: {sub_dirs[:5]} ... {sub_dirs[-1]}")

dims_found = set()
consistent = True

for sub_dir in sub_dirs:
    dir_path = os.path.join(base_path, sub_dir)
    
    # Lấy danh sách file npy trong folder này
    files = [f for f in os.listdir(dir_path) if f.endswith('.npy')]
    
    if not files:
        print(f"[CẢNH BÁO] Thư mục {sub_dir} rỗng!")
        continue
        
    # Lấy ngẫu nhiên 1 file để kiểm tra
    sample_file = random.choice(files)
    file_path = os.path.join(dir_path, sample_file)
    
    try:
        data = np.load(file_path)
        feature_dim = data.shape[-1]
        dims_found.add(feature_dim)
        
        # Chỉ in chi tiết cho thư mục đầu tiên để đỡ rối
        if sub_dir == '1':
            print(f"\n--- Kiểm tra chi tiết thư mục {sub_dir} ---")
            print(f"File mẫu: {sample_file}")
            print(f"Shape: {data.shape}")
            print(f"Số chiều: {feature_dim}")
            
    except Exception as e:
        print(f"Lỗi đọc file {file_path}: {e}")

print("\n" + "="*40)
print("KẾT QUẢ KIỂM TRA TOÀN BỘ")
print("="*40)

if len(dims_found) == 1:
    dim = dims_found.pop()
    print(f"✅ DỮ LIỆU NHẤT QUÁN. Toàn bộ dataset có số chiều: {dim}")
    
    if dim == 171:
        print("\n=> KẾT LUẬN: Dữ liệu của bạn chỉ có Position (57 khớp x 3).")
        print("   1. Code tính index 'joint_idx * 3' của tôi là ĐÚNG.")
        print("   2. Bạn CẦN sửa file 'vq_diffusion.py': đặt dim_pose = 171")
        print("   3. Đừng quên truyền hand_indices, body_indices vào model.")
        
    elif dim == 264:
        print("\n=> KẾT LUẬN: Dữ liệu chuẩn BEAT đầy đủ (264 chiều).")
        print("   1. Code tính index cũ của tôi KHÔNG dùng được.")
        print("   2. Bạn cần giữ nguyên 'dim_pose = 264'.")
        print("   3. Báo lại tôi để tôi viết hàm map index mới cho chuẩn 264.")
        
    else:
        print(f"\n=> KẾT LUẬN: Số chiều lạ ({dim}). Cần kiểm tra lại pipeline xử lý dữ liệu.")
else:
    print(f"❌ DỮ LIỆU KHÔNG NHẤT QUÁN! Tìm thấy các số chiều khác nhau: {dims_found}")
    print("=> Bạn cần xử lý lại dữ liệu để đảm bảo đồng bộ.")

Tìm thấy 30 thư mục con: ['1', '2', '3', '4', '5'] ... 30

--- Kiểm tra chi tiết thư mục 1 ---
File mẫu: 1_wayne_0_49_49_sentence_002.npy
Shape: (465, 264)
Số chiều: 264

KẾT QUẢ KIỂM TRA TOÀN BỘ
✅ DỮ LIỆU NHẤT QUÁN. Toàn bộ dataset có số chiều: 264

=> KẾT LUẬN: Dữ liệu chuẩn BEAT đầy đủ (264 chiều).
   1. Code tính index cũ của tôi KHÔNG dùng được.
   2. Bạn cần giữ nguyên 'dim_pose = 264'.
   3. Báo lại tôi để tôi viết hàm map index mới cho chuẩn 264.


### Visual data bằng mean/std chuẩn tính từ bộ dữ liệu

In [17]:
%matplotlib inline
import matplotlib.pyplot as plt

import sys
import os
import numpy as np
import joblib

# Setup path
PYOM_DIR = "/home/serverai/ltdoanh/Motion_Diffusion/datasets/pymo"
if PYOM_DIR not in sys.path:
    sys.path.insert(0, PYOM_DIR)

from pymo.parsers import BVHParser
from pymo.preprocessing import *
from pymo.viz_tools import draw_stickfigure, draw_stickfigure3d
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

print("✅ Imports successful!")

# ===== CELL 2: Load Pipeline =====
pipeline_path = "/home/serverai/ltdoanh/Motion_Diffusion/global_pipeline.pkl"

print(f"📦 Loading pipeline from: {pipeline_path}")

pipeline = joblib.load(pipeline_path)
print(f"✅ Pipeline loaded!")
print(f"   Mean shape: {pipeline.named_steps['stdscale'].data_mean_.shape}, First 5 values: {pipeline.named_steps['stdscale'].data_mean_[:5]}")
print(f"   Std shape: {pipeline.named_steps['stdscale'].data_std_.shape},  First 5 values: {pipeline.named_steps['stdscale'].data_std_[:5]}")

# ===== CELL 3: Load and Visualize Motion =====
npy_path = "/home/serverai/ltdoanh/Motion_Diffusion/datasets/BEAT_numpy/npy/1/1_wayne_0_1_1_sentence_010.npy"
frame = 50

print(f"\n🎬 Visualizing: {os.path.basename(npy_path)}")
print(f"   Frame: {frame}")

# Load motion data
motion_data = np.load(npy_path)
print(f"   Motion shape: {motion_data.shape}")

# Inverse transform
print("   Performing inverse transform...")
reconstructed = pipeline.inverse_transform([motion_data])
print(f"   ✅ Reconstructed shape: {reconstructed[0].values.shape}")

# Visualize 2D
print("\n   Creating 2D visualization...")
fig1 = plt.figure(figsize=(10, 8))
draw_stickfigure(reconstructed[0], frame=frame)
plt.title(f"2D Stick Figure - Frame {frame}")
plt.tight_layout()
plt.show()

print("   ✅ 2D plot displayed!")

# Visualize 3D
print("\n   Creating 3D visualization...")
fig2 = plt.figure(figsize=(10, 8))
draw_stickfigure3d(reconstructed[0], frame=frame)
plt.title(f"3D Stick Figure - Frame {frame}")
plt.tight_layout()
plt.show()

print("   ✅ 3D plot displayed!")

✅ Imports successful!
📦 Loading pipeline from: /home/serverai/ltdoanh/Motion_Diffusion/global_pipeline.pkl
✅ Pipeline loaded!
   Mean shape: (264,), First 5 values: [-0.05385426  8.01535389 -3.75700843  0.02310581 18.43449509]
   Std shape: (264,),  First 5 values: [1.17785947 2.32203438 1.24202695 1.89584987 2.06571074]

🎬 Visualizing: 1_wayne_0_1_1_sentence_010.npy
   Frame: 50
   Motion shape: (261, 264)
   Performing inverse transform...


AttributeError: 'Numpyfier' object has no attribute 'org_mocap_'

### Visual data bằng mean/std từ model được huấn luyện - Latent MotionDiffuse

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import sys
import os
import numpy as np
import joblib
import torch

# Setup path
PYOM_DIR = "/home/serverai/ltdoanh/Motion_Diffusion/datasets/pymo"
if PYOM_DIR not in sys.path:
    sys.path.insert(0, PYOM_DIR)

from pymo.parsers import BVHParser
from pymo.preprocessing import *
from pymo.viz_tools import draw_stickfigure, draw_stickfigure3d
from sklearn.pipeline import Pipeline

print("✅ Imports successful!")

# ===== CREATE PIPELINE WITH META =====
print("\n🔨 Creating pipeline from scratch...")

# 1. Create empty pipeline
pipeline = Pipeline([
    ('param', MocapParameterizer('position')),
    ('rcpn', RootCentricPositionNormalizer()),
    ('delta', RootTransformer('abdolute_translation_deltas')),
    ('const', ConstantsRemover()),
    ('np', Numpyfier()),
    ('down', DownSampler(2)),
    ('stdscale', ListStandardScaler())
])

# 2. Fit pipeline on sample BVH (to learn structure)
bvh_sample_path = "/home/serverai/ltdoanh/Motion_Diffusion/datasets/BEAT/1/1_wayne_0_1_1.bvh"
print(f"   Fitting on BVH sample: {os.path.basename(bvh_sample_path)}")

parser = BVHParser()
parsed_data = parser.parse(bvh_sample_path)
pipeline.fit([parsed_data])
print("   ✅ Pipeline fitted (structure learned)")

# 3. Load mean/std from meta directory
# meta_dir = "/home/serverai/ltdoanh/Motion_Diffusion/checkpoints/beat/test/meta"
# mean_path = os.path.join(meta_dir, "mean.npy")
# std_path = os.path.join(meta_dir, "std.npy")
ckpt_path = "/home/serverai/ltdoanh/Motion_Diffusion/checkpoints/beat/vq_diffusion/model/latest.pt"  # Thay đường dẫn file của bạn vào đây

try:
    # 2. Load checkpoint
    # map_location='cpu' giúp tránh lỗi nếu máy bạn không có GPU giống lúc train
    checkpoint = torch.load(ckpt_path, map_location='cpu', weights_only=False)

    # 3. Kiểm tra và lấy mean, std
    if 'mean' in checkpoint and 'std' in checkpoint:
        mean = checkpoint['mean']
        std = checkpoint['std']

        print("--- Đã tìm thấy Mean và Std ---")
        print(f"Shape của Mean: {mean.shape}")
        print(f"Shape của Std: {std.shape}")
        
        # In thử vài giá trị đầu
        print(f"Mean (5 giá trị đầu): {mean[:5]}")
        print(f"Std (5 giá trị đầu): {std[:5]}")
        
        # 4. (Tùy chọn) Lưu lại ra file .npy để dùng việc khác nếu cần
        # np.save('mean.npy', mean)
        # np.save('std.npy', std)
        # print("Đã lưu ra file .npy")
        
    else:
        print("Không tìm thấy key 'mean' hoặc 'std' trong file .pt này.")
        print("Các keys hiện có:", checkpoint.keys())

except Exception as e:
    print(f"Có lỗi xảy ra: {e}")

# 4. Override pipeline's mean/std with meta values
print("\n🔧 Overriding pipeline statistics with meta values...")
pipeline.named_steps['stdscale'].data_mean_ = mean
pipeline.named_steps['stdscale'].data_std_ = std
print("   ✅ Pipeline updated with meta statistics!")

# ===== VISUALIZE MOTION =====
npy_path = "/home/serverai/ltdoanh/Motion_Diffusion/datasets/BEAT_numpy/npy/2/2_scott_0_2_2_sentence_000.npy"
frame = 50

print(f"\n🎬 Visualizing: {os.path.basename(npy_path)}")
print(f"   Frame: {frame}")

# Load motion data
motion_data = np.load(npy_path)
print(f"   Motion shape: {motion_data.shape}")

# Inverse transform
print("   Performing inverse transform...")
reconstructed = pipeline.inverse_transform([motion_data])
print(f"   ✅ Reconstructed shape: {reconstructed[0].values.shape}")

# Visualize 2D
print("\n   Creating 2D visualization...")
fig1 = plt.figure(figsize=(10, 8))
draw_stickfigure(reconstructed[0], frame=frame)
plt.title(f"2D Stick Figure (Meta) - Frame {frame}")
plt.tight_layout()
plt.show()

print("   ✅ 2D plot displayed!")

# Visualize 3D
print("\n   Creating 3D visualization...")
fig2 = plt.figure(figsize=(10, 8))
draw_stickfigure3d(reconstructed[0], frame=frame)
plt.title(f"3D Stick Figure (Meta) - Frame {frame}")
plt.tight_layout()
plt.show()

print("   ✅ 3D plot displayed!")

✅ Imports successful!

🔨 Creating pipeline from scratch...
   Fitting on BVH sample: 1_wayne_0_1_1.bvh


FileNotFoundError: [Errno 2] No such file or directory: '/home/serverai/ltdoanh/Motion_Diffusion/datasets/BEAT/1/1_wayne_0_1_1.bvh'

### Visual data bằng mean/std từ model được huấn luyện - MotionDiffuse

In [ ]:
import torch
from models import MotionTransformer
from utils.get_opt import get_opt
import numpy as np
from os.path import join as pjoin
from trainers import DDPMTrainer

encoder = MotionTransformer(
        input_feats=264,
        num_frames=360,
        num_layers=8,
        latent_dim=512,
        no_clip=False,
        no_eff=False)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
opt_path = "/home/serverai/ltdoanh/Motion_Diffusion/checkpoints/beat/test/opt.txt"
opt = get_opt(opt_path, device)
opt.do_denoise = True
encoder.to(opt.device)
mean = np.load(pjoin(opt.meta_dir, 'mean.npy'))
std = np.load(pjoin(opt.meta_dir, 'std.npy'))
trainer = DDPMTrainer(opt, encoder)
trainer.load(pjoin('/home/serverai/ltdoanh/Motion_Diffusion/checkpoints/beat/test/model/ckpt_e045.tar'))

trainer.eval_mode()


result_dict = {}
with torch.no_grad():
        caption = ["the first thing i like to do on weekends is relaxing and i'll go shopping if i'm not that tired"]
        m_lens = torch.LongTensor([64]).to(device)
        pred_motions = trainer.generate(caption, m_lens, 264)
        motion = pred_motions[0].cpu().numpy()
        print(motion.shape)
opt.result_dir = "/home/serverai/ltdoanh/Motion_Diffusion/results"
np.save(pjoin(opt.result_dir, 'motion_v0.npy'), motion)

Reading /home/serverai/ltdoanh/Motion_Diffusion/checkpoints/beat/test/opt.txt


100%|██████████| 1000/1000 [00:16<00:00, 58.97it/s]

(64, 264)


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import sys
import os
import numpy as np
import joblib
import torch

# Setup path
PYOM_DIR = "/home/serverai/ltdoanh/Motion_Diffusion/datasets/pymo"
if PYOM_DIR not in sys.path:
    sys.path.insert(0, PYOM_DIR)

from pymo.parsers import BVHParser
from pymo.preprocessing import *
from pymo.viz_tools import draw_stickfigure, draw_stickfigure3d
from sklearn.pipeline import Pipeline

print("✅ Imports successful!")

# ===== 1. CREATE PIPELINE WITH META =====
print("\n🔨 Creating pipeline from scratch...")

pipeline = Pipeline([
    ('param', MocapParameterizer('position')),
    ('rcpn', RootCentricPositionNormalizer()),
    ('delta', RootTransformer('abdolute_translation_deltas')),
    ('const', ConstantsRemover()),
    ('np', Numpyfier()),
    ('down', DownSampler(2)),
    ('stdscale', ListStandardScaler())
])

# ===== 2. FIT PIPELINE (Structure Learning) =====
# Cần fit vào một file BVH mẫu để pipeline học được cấu trúc xương (skeleton)
bvh_sample_path = "/home/serverai/ltdoanh/Motion_Diffusion/datasets/2_scott_0_55_55.bvh"
print(f"   Fitting on BVH sample: {os.path.basename(bvh_sample_path)}")

if os.path.exists(bvh_sample_path):
    parser = BVHParser()
    parsed_data = parser.parse(bvh_sample_path)
    pipeline.fit([parsed_data])
    print("   ✅ Pipeline fitted (structure learned)")
else:
    print(f"❌ Error: BVH file not found at {bvh_sample_path}")
    sys.exit(1)

# ===== 3. LOAD MEAN/STD FROM META (.npy files) =====
meta_dir = "/home/serverai/ltdoanh/Motion_Diffusion/checkpoints/beat/test/meta"
mean_path = os.path.join(meta_dir, "mean.npy")
std_path = os.path.join(meta_dir, "std.npy")

print("\n📥 Loading Mean/Std from Meta files...")
try:
    # Load trực tiếp từ file .npy
    mean = np.load(mean_path)
    std = np.load(std_path)

    print("   ✅ Đã tìm thấy Mean và Std từ file .npy")
    print(f"   Shape của Mean: {mean.shape}")
    print(f"   Shape của Std: {std.shape}")
    
    # In thử vài giá trị đầu để kiểm tra
    print(f"   Mean (5 giá trị đầu): {mean[:5]}")
    
    # Override pipeline statistics
    print("\n🔧 Overriding pipeline statistics with meta values...")
    pipeline.named_steps['stdscale'].data_mean_ = mean
    pipeline.named_steps['stdscale'].data_std_ = std
    print("   ✅ Pipeline updated with meta statistics!")

except Exception as e:
    print(f"❌ Có lỗi xảy ra khi load meta: {e}")
    # Nếu không có mean/std thì visualization sẽ bị sai, nên dừng hoặc cảnh báo
    sys.exit(1)

In [ ]:
# ===== 4. VISUALIZE MOTION =====
npy_path = "/home/serverai/ltdoanh/Motion_Diffusion/results/motion_v0.npy"
frame = 0

print(f"\n🎬 Visualizing: {os.path.basename(npy_path)}")

if os.path.exists(npy_path):
    # Load motion data
    motion_data = np.load(npy_path)
    print(f"   Motion shape: {motion_data.shape}")

    # Inverse transform
    print("   Performing inverse transform...")
    # Lưu ý: inverse_transform trả về list các object MocapData
    reconstructed = pipeline.inverse_transform([motion_data])
    print(f"   ✅ Reconstructed shape: {reconstructed[0].values.shape}")

    # Visualize 2D
    print("\n   Creating 2D visualization...")
    fig1 = plt.figure(figsize=(10, 8))
    draw_stickfigure(reconstructed[0], frame=frame)
    plt.title(f"2D Stick Figure (Meta Mean/Std) - Frame {frame}")
    plt.tight_layout()
    plt.show()

    # Visualize 3D
    print("\n   Creating 3D visualization...")
    fig2 = plt.figure(figsize=(10, 8))
    draw_stickfigure3d(reconstructed[0], frame=frame)
    plt.title(f"3D Stick Figure (Meta Mean/Std) - Frame {frame}")
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    print("   ✅ Done!")
else:
    print(f"❌ Error: Motion file not found at {npy_path}")

### Inference 

In [ ]:
import torch
import numpy as np
import os
import sys

from models import MotionTransformer
from trainers import DDPMTrainer
from models.vq.model import RVQVAE 

# ==========================================
# 1. Cấu hình Inference
# ==========================================
class InferenceConfig:
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

        self.is_train = False
        self.schedule_sampler = 'uniform'   

        # --- Cấu hình Diffusion ---
        self.input_feats = 512     # Latent Dimension
        self.num_frames = 45      # Latent Length (360 / 8)
        self.num_layers = 8
        self.latent_dim = 512
        self.ff_size = 1024
        self.num_heads = 8
        self.dropout = 0.1
        self.activation = "gelu"
        self.dataset_name = 'beat' 
        self.do_denoise = True
        self.noise_schedule = 'cosine'
        self.diffusion_steps = 1000
        self.no_clip = False
        self.no_eff = False
        self.result_dir = "/home/serverai/ltdoanh/Motion_Diffusion/results"

# Class giả lập args cho RVQVAE
class VQArgs:
    def __init__(self):
        # Các giá trị mặc định, sẽ được cập nhật logic bên dưới
        self.num_quantizers = 1 
        self.shared_codebook = False
        self.quantize_dropout_prob = 0.0
        self.mu = 0.99 # Cho QuantizerEMA

opt = InferenceConfig()
vq_args = VQArgs()

# ==========================================
# 2. Load Checkpoint & Tách Weights
# ==========================================
ckpt_path = "/home/serverai/ltdoanh/Motion_Diffusion/checkpoints/beat/vqkl_diffusion_hierarchical/model/best_model.pt"
print(f"📂 Loading checkpoint: {ckpt_path}")

# Load toàn bộ checkpoint
checkpoint = torch.load(ckpt_path, map_location='cpu', weights_only=False)

state_dict = checkpoint['model_state_dict']

# DEBUG: In ra một vài key để kiểm tra prefix thực tế
print("🔍 Sample keys in checkpoint:")
for i, k in enumerate(list(state_dict.keys())[:5]):
    print(f"   {k}")

# --- Tự động phát hiện số lượng Quantizers từ Checkpoint ---
max_layer_idx = 0
found_quantizer = False
for k in state_dict.keys():
    # FIX: Check prefix 'vqkl.' instead of 'vqvae.'
    if "quantizer.layers." in k: 
        found_quantizer = True
        try:
            # Tìm số nằm sau 'layers.'
            parts = k.split('.')
            layer_idx = int(parts[parts.index('layers') + 1])
            if layer_idx > max_layer_idx:
                max_layer_idx = layer_idx
        except:
            pass

if found_quantizer:
    vq_args.num_quantizers = max_layer_idx + 1
else:
    print("⚠️ Warning: Could not detect quantizer layers. Defaulting to 1.")
    vq_args.num_quantizers = 1
    
print(f"🔍 Detected num_quantizers: {vq_args.num_quantizers}")

# --- Tách Dictionary ---
trans_dict = {}
vqvae_dict = {}

for k, v in state_dict.items():
    if k.startswith('transformer.'):
        trans_dict[k[12:]] = v  
    # FIX: Prefix là 'vqkl.' (5 chars) chứ không phải 'vqvae.' (6 chars)
    elif k.startswith('vqkl.'):
        vqvae_dict[k[5:]] = v   

print(f"📊 Loaded {len(trans_dict)} keys for Transformer")
print(f"📊 Loaded {len(vqvae_dict)} keys for RVQVAE")

# A. Motion Transformer
print("🔧 Initializing MotionTransformer...")
encoder = MotionTransformer(
    input_feats=opt.input_feats,
    num_frames=opt.num_frames,
    num_layers=opt.num_layers,
    latent_dim=opt.latent_dim,
    num_heads=opt.num_heads,
    ff_size=opt.ff_size,
    no_clip=opt.no_clip,
    no_eff=opt.no_eff
)
encoder.load_state_dict(trans_dict, strict=True)
encoder.to(opt.device).eval()

# B. RVQVAE
print("🔧 Initializing RVQVAE...")
# Lưu ý: Các tham số dưới đây phải khớp với file config lúc train VQVAE của bạn.
# Tôi đang để các giá trị phổ biến dựa trên file model.py
vqvae_model = RVQVAE(
    args=vq_args,
    input_width=264,       # BEAT dataset thường là 264
    nb_code=1024,          # FIX: Updated from 512 to 1024 to match checkpoint
    code_dim=512, 
    output_emb_width=512, 
    down_t=3, 
    stride_t=2, 
    width=512, 
    depth=3, 
    dilation_growth_rate=3,
    activation='relu',
    norm=None,
    embed_dim=512,         # FIX: Added for VQ-KL (matches checkpoint)
    double_z=True          # FIX: Enable VQ-KL mode (loads quant_conv/post_quant_conv)
)
vqvae_model.load_state_dict(vqvae_dict, strict=True)
vqvae_model.to(opt.device).eval()

# ==========================================
# 4. Inference
# ==========================================
trainer = DDPMTrainer(opt, encoder)

# Inject mean/std (Quan trọng cho quá trình decode cuối cùng)
trainer.mean = checkpoint['mean']
trainer.std = checkpoint['std']

print("🚀 Starting Inference...")
os.makedirs(opt.result_dir, exist_ok=True)

with torch.no_grad():
    caption = ["the first thing i like to do on weekends is relaxing and i'll go shopping if i'm not that tired"]
    
    # Độ dài Latent (45)
    m_lens = torch.LongTensor([45]).to(opt.device) 
    
    # 1. Sinh Latent (Diffusion) -> Output: (Batch, Length, Dim) = (1, 45, 512)
    pred_latent_list = trainer.generate(caption, m_lens, dim_pose=512)
    pred_latent = pred_latent_list[0]

    if pred_latent.dim() == 2:
        pred_latent = pred_latent.unsqueeze(0)  # Thêm batch dim nếu cần

    print(f"   Latent generated shape: {pred_latent.shape}")

    # 2. Decode bằng RVQVAE
    # RVQVAE Decoder cần input: (Batch, Channel, Length) -> Cần permute
    latent_input = pred_latent.permute(0, 2, 1) # -> (1, 512, 45)
    
    print("   Decoding with RVQVAE...")
    # Gọi trực tiếp decoder (bỏ qua quantizer vì Diffusion đã sinh ra latent rồi)
    decoded_motion = vqvae_model.decoder(latent_input)
    
    # 3. Post-process (Permute lại về: Batch, Length, Channel)
    # Hàm postprocess trong model.py: (B, C, T) -> (B, T, C)
    motion = vqvae_model.postprocess(decoded_motion).cpu().numpy()

    if motion.shape[1] == 264 and motion.shape[2] == 360:
        motion = motion.transpose(0, 2, 1)

    # motion = motion.cpu().numpy()
    
    # 4. Denormalize (Giải chuẩn hóa)
    # Output của VQVAE thường vẫn là normalized data
    mean = checkpoint['mean']
    std = checkpoint['std']
    
    # Đảm bảo shape khớp để broadcast
    # Motion: (1, 360, 264), Mean: (264,), Std: (264,)
    motion = motion * std + mean
    
    print(f"🎉 Final Motion Shape: {motion.shape}")

# Lưu kết quả
save_path = os.path.join(opt.result_dir, 'inference-latest.npy')
np.save(save_path, motion)
print(f"💾 Saved to: {save_path}")

📂 Loading checkpoint: /home/serverai/ltdoanh/Motion_Diffusion/checkpoints/beat/vqkl_diffusion_hierarchical/model/best_model.pt
🔍 Sample keys in checkpoint:
   vqkl.encoder.model.0.weight
   vqkl.encoder.model.0.bias
   vqkl.encoder.model.2.0.weight
   vqkl.encoder.model.2.0.bias
   vqkl.encoder.model.2.1.model.0.conv1.weight
🔍 Detected num_quantizers: 10
📊 Loaded 685 keys for Transformer
📊 Loaded 128 keys for RVQVAE
🔧 Initializing MotionTransformer...
🔧 Initializing RVQVAE...
🚀 Starting Inference...


100%|██████████| 1000/1000 [00:17<00:00, 57.58it/s]


   Latent generated shape: torch.Size([1, 45, 512])
   Decoding with RVQVAE...
🎉 Final Motion Shape: (1, 360, 264)
💾 Saved to: /home/serverai/ltdoanh/Motion_Diffusion/results/inference-latest.npy


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import sys
import os
import numpy as np
import joblib
import torch

# Setup path
PYOM_DIR = "/home/serverai/ltdoanh/Motion_Diffusion/datasets/pymo"
if PYOM_DIR not in sys.path:
    sys.path.insert(0, PYOM_DIR)

from pymo.parsers import BVHParser
from pymo.preprocessing import *
from pymo.viz_tools import draw_stickfigure, draw_stickfigure3d
from sklearn.pipeline import Pipeline

print("✅ Imports successful!")

# ===== CREATE PIPELINE WITH META =====
print("\n🔨 Creating pipeline from scratch...")

# 1. Create empty pipeline
pipeline = Pipeline([
    ('param', MocapParameterizer('position')),
    ('rcpn', RootCentricPositionNormalizer()),
    ('delta', RootTransformer('abdolute_translation_deltas')),
    ('const', ConstantsRemover()),
    ('np', Numpyfier()),
    ('down', DownSampler(2)),
    ('stdscale', ListStandardScaler())
])

# 2. Fit pipeline on sample BVH (to learn structure)
bvh_sample_path = "/home/serverai/ltdoanh/Motion_Diffusion/datasets/BEAT/1/1_wayne_0_1_1.bvh"
print(f"   Fitting on BVH sample: {os.path.basename(bvh_sample_path)}")

parser = BVHParser()
parsed_data = parser.parse(bvh_sample_path)
pipeline.fit([parsed_data])
print("   ✅ Pipeline fitted (structure learned)")

# 3. Load mean/std from meta directory
ckpt_path = "/home/serverai/ltdoanh/Motion_Diffusion/checkpoints/beat/vqkl_diffusion/model/best_model.pt"  

try:
    # 2. Load checkpoint
    # map_location='cpu' giúp tránh lỗi nếu máy bạn không có GPU giống lúc train
    checkpoint = torch.load(ckpt_path, map_location='cpu', weights_only=False)

    # 3. Kiểm tra và lấy mean, std
    if 'mean' in checkpoint and 'std' in checkpoint:
        mean = checkpoint['mean']
        std = checkpoint['std']

        print("--- Đã tìm thấy Mean và Std ---")
        print(f"Shape của Mean: {mean.shape}")
        print(f"Shape của Std: {std.shape}")
        
        # In thử vài giá trị đầu
        print(f"Mean (5 giá trị đầu): {mean[:5]}")
        print(f"Std (5 giá trị đầu): {std[:5]}")
        
    else:
        print("Không tìm thấy key 'mean' hoặc 'std' trong file .pt này.")
        print("Các keys hiện có:", checkpoint.keys())

except Exception as e:
    print(f"Có lỗi xảy ra: {e}")

# 4. Override pipeline's mean/std with meta values
print("\n🔧 Overriding pipeline statistics with meta values...")
pipeline.named_steps['stdscale'].data_mean_ = mean
pipeline.named_steps['stdscale'].data_std_ = std
print("   ✅ Pipeline updated with meta statistics!")

# ===== VISUALIZE MOTION =====
npy_path = "/home/serverai/ltdoanh/Motion_Diffusion/results/inference-latest.npy"
frame = 50

print(f"\n🎬 Visualizing: {os.path.basename(npy_path)}")
print(f"   Frame: {frame}")

# Load motion data
motion_data = np.load(npy_path)
print(f"   Motion shape: {motion_data.shape}")
if motion_data.ndim == 3:
    motion_data = motion_data[0] # Lấy mẫu đầu tiên -> (360, 264)
    print(f"   Squeezed Motion shape: {motion_data.shape}")

# Inverse transform
print("   Performing inverse transform...")
reconstructed = pipeline.inverse_transform([motion_data])
print(f"   ✅ Reconstructed shape: {reconstructed[0].values.shape}")

# Visualize 2D
print("\n   Creating 2D visualization...")
fig1 = plt.figure(figsize=(10, 8))
draw_stickfigure(reconstructed[0], frame=frame)
plt.title(f"2D Stick Figure (Meta) - Frame {frame}")
plt.tight_layout()
plt.show()

print("   ✅ 2D plot displayed!")

# Visualize 3D
print("\n   Creating 3D visualization...")
fig2 = plt.figure(figsize=(10, 8))
draw_stickfigure3d(reconstructed[0], frame=frame)
plt.title(f"3D Stick Figure (Meta) - Frame {frame}")
plt.tight_layout()
plt.show()

print("   ✅ 3D plot displayed!")

### Inference CFG